In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [134]:
import pandas as pd
import numpy as np
from numpy import dot
from numpy.linalg import norm
import datetime as dt
import math as mt
from pymongo import MongoClient as mc
import IPython
import IPython.display
from sklearn.decomposition import TruncatedSVD 
from scipy.spatial import distance
import scipy.stats
from modules.models import *
from modules.utils import *
import modules.new_utils as nu
from tqdm import trange

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sn
sn.set()
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# 한글출력
matplotlib.rc('font', family='AppleGothic')  # MacOS
# matplotlib.rc('font', family='Malgun Gothic')  # Windows
plt.rcParams['axes.unicode_minus'] = False

In [3]:
# 1. read datas.xlsx
xlsx = load_excel("datas/datas.xlsx")

In [15]:
# 2. data pre-processing
p, m = data_preprocessing(xlsx)

In [82]:
# 3. bill - calc
min_per = 10
max_per = 80
bc_result = nu.bill_calc(m, p, min_per, max_per)

1 월 계산 진행 합니다.
2 월 계산 진행 합니다.
3 월 계산 진행 합니다.
4 월 계산 진행 합니다.
5 월 계산 진행 합니다.
6 월 계산 진행 합니다.
7 월 계산 진행 합니다.
8 월 계산 진행 합니다.
9 월 계산 진행 합니다.
10 월 계산 진행 합니다.
11 월 계산 진행 합니다.
12 월 계산 진행 합니다.


In [87]:
# 4. normal_analysis
# 일반분석
# - 각 월 별 유리 가구 수, 아파트 전체 사용요금, 공동설비사용요금, 손실율에서 종합계약이 언제부터 유리해지는 지 확인할 수 있다.
na_result = nu.normal_analysis(bc_result)

In [120]:
# 추천이 들어가는 곳
# 5. mean_analysis
# 평균치를 기준으로 사용량을 재구성하여 일반분석 진행
mean_result = nu.mean_analysis(m, p, min_per, max_per)

1 월 계산 진행 합니다.


In [122]:
mean_result[1]

{'positive_count': {'comp':     10   11    12    13    14    15    16    17    18    19  ...     71  \
  1  8.0  8.0  10.0  13.0  13.0  15.0  23.0  20.0  27.0  30.0  ...  398.0   
  
        72     73     74     75     76     77     78     79     80  
  1  398.0  398.0  398.0  398.0  398.0  398.0  398.0  398.0  398.0  
  
  [1 rows x 71 columns],
  'single':       10     11     12     13     14     15     16     17     18     19  ...  \
  1  390.0  390.0  388.0  385.0  385.0  383.0  375.0  378.0  371.0  368.0  ...   
  
      71   72   73   74   75   76   77   78   79   80  
  1  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
  
  [1 rows x 71 columns]},
 'loss_ratio': {'comp':       10     11     12     13     14     15     16     17     18     19  ...  \
  1  118.0  117.0  117.0  116.0  116.0  115.0  114.0  114.0  113.0  113.0  ...   
  
       71    72    73    74    75    76    77    78    79    80  
  1  64.0  63.0  63.0  62.0  61.0  61.0  60.0  59.0  59.0  58.0  
  
  [1 rows

In [167]:
a = m.set_index("month")
test = np.array(a)
r,c = test.shape
imp_weight = 0.99

recos = np.array([])

for t in trange(c - 1):
    if t == 0:
        dr_matrix = test
    else:
        dr_size = c - t
        svd = TruncatedSVD(n_components=dr_size)
        dr_matrix = svd.fit_transform(test)
    
    imp = np.array([])
    for A in dr_matrix:
        _ = np.array([])
        for B in dr_matrix:
            _ = np.append(_, improved_similarity(
                    A, B, imp_weight
                ))
        imp = np.append(imp, _)
        imp = imp.reshape(r, -1)
        
    reco_idx = imp.mean(axis=1).argmax()
    
    if ~np.any(recos == reco_idx):
        recos = np.append(recos, reco_idx)

100%|██████████| 397/397 [00:03<00:00, 112.36it/s]


In [168]:
recos

array([ 1.,  2.,  9., 10.])